In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
import matplotlib.pyplot as plt
if tf.config.experimental.list_physical_devices('GPU'):
    print('GPU is used')
else:
    print('no GPU available, CPU is used instead')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

GPU is used


In [4]:
np.set_printoptions(precision=4)

In [5]:
#Read in the datasets for MITBIH
df_train= pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)
df_test=pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv',header=None)
print("Dataframes MITBIH correctly read into workspace")

#split target and value
train_target=df_train[187]
test_target=df_test[187]
train=df_train.drop(187,axis=1)
test=df_test.drop(187,axis=1)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/heartbeat/mitbih_train.csv'

In [ ]:
#Switches to decide the dataset sampling method and which models should be run
class Config_Sampling:
    oversample = True #equals to B_SMOTE
    undersample = False
    sample_name = "UNDEFINED_SAMPLE"
    
    

Train_Simple_ANN = False #Trains the simple ANN
Train_Simple_CNN = False #Trains the simple CNN
Train_Advanced_CNN = True #Trains the advanced CNN
 

In [ ]:
oversampler = SMOTE()
undersampler = RandomUnderSampler()

In [ ]:
#Based on the configuration in the Config_Sampling Class, the datasets are sampled and the sample name is modified accordingly
if Config_Sampling.oversample:
    train, train_target = oversampler.fit_resample(df_train.iloc[:,:-1], df_train.iloc[:,-1])
    Config_Sampling.sample_name = "MITBIH_B_SMOTE"
    print("Sample Name:", Config_Sampling.sample_name)
elif Config_Sampling.undersample:
    train, train_target = undersampler.fit_resample(df_train.iloc[:,:-1], df_train.iloc[:,-1])
    Config_Sampling.sample_name = "MITBIH_C_RUS"
    print("Sample Name:", Config_Sampling.sample_name)
else: 
    print("Using the original mitbih dataset")
    Config_Sampling.sample_name = "MITBIH_A_Original"
    print("Sample Name:", Config_Sampling.sample_name)

# **Deep Learning Models**

## **Simple Artificial neural Network**
ANN without convolutional layers. Only Dense layers are used. No Pooling, Flattening or Dropping out. Base model for later comparison.

In [ ]:
if Train_Simple_ANN == True:
    class Config_ANN:
        epochs = 70 #70 is default (exp1)
        batch_size = 10 #10 is default (exp1)
        exp_name = 4 #Experiment Number counter 
        patience = 70 #10 # is default (exp1)
        initial_learning_rate=0.001 #Default Initial Learning Rate for ADAM: 0.001
        optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
        reduce_lr_every_10_epochs = True #Reduce the learning rate each 10 Epochs with the rate defined below. Default is False for exp1.
        lr_reduction_rate = 0.5 # Reduction Rate for reducing each 10 epochs. 0.5 means, that lr is halfed each 10 epochs.
        filepath_checkpoint = '/kaggle/working/Simple_ANN/'+str(Config_Sampling.sample_name) +'/experiment_'+str(exp_name) + '_'+str (Config_Sampling.sample_name) +'.weights.h5'
        filepath_accuracy_plot = '/kaggle/working/Simple_ANN/'+str(Config_Sampling.sample_name) +'/experiment_'+str(exp_name) +  '_'+ str (Config_Sampling.sample_name) +'.accuracy_plot.png'
        filepath_loss_plot = '/kaggle/working/Simple_ANN/'+str(Config_Sampling.sample_name) +'/experiment_'+str(exp_name) +  '_'+str (Config_Sampling.sample_name) +'.loss_plot.png'
        filepath_classification_report = '/kaggle/working/Simple_ANN/'+str(Config_Sampling.sample_name) +'/experiment_'+str(exp_name) +  '_'+str (Config_Sampling.sample_name) +'.classification_report.txt'

    #Model structure: This is not changed during experiments.
    ann_model = tf.keras.models.Sequential()
    ann_model.add(tf.keras.layers.Dense(60, activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape=(187,)))
    ann_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    ann_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    ann_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    ann_model.add(tf.keras.layers.Dense(5, activation='softmax'))

    #Function for the learning rate scheduler
    def lr_scheduler(epoch, lr):
        if Config_ANN.reduce_lr_every_10_epochs and epoch % 10 == 0: # %10 == 0 means each 10 epochs (only then no "rest" after dividing trough 10)
            return lr * Config_ANN.lr_reduction_rate  # reduce the learning rate with the configured rate
        else:
            return lr

    # callback for the learning rate reduction schedule (function)
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

    #Model checkpoint: Saves the best model for all epochs with regards to the validation accuracy. Only weights (.h5) are saved.
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=Config_ANN.filepath_checkpoint,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    #Early stop callback: If validation accuracy does not change during the last XXX epochs, training is stopped (XXX is configured as patience)
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=Config_ANN.patience)

    #Compilation of model with ADAM and custom lr, sparse_categorical_crossentropy since we have integers as class labels.
    ann_model.compile(optimizer=Config_ANN.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    ann_model.summary() #Model summary for usage in reports or other files.

    #training the model and storing all results in training history. aside from the datasets, all arguments are called from our config-class. Also calling the respective callbacks.
    ann_model_history = ann_model.fit(train, train_target, epochs=Config_ANN.epochs, batch_size = Config_ANN.batch_size, validation_data = (test, test_target), callbacks=[model_checkpoint_callback, early_stop_callback, lr_scheduler_callback])

    #some plots for early interpretion during the run
    plt.plot(ann_model_history.history['accuracy'])
    plt.plot(ann_model_history.history['val_accuracy'])
    plt.legend(["accuracy","val_accuracy"])
    plt.title('Accuracy Vs Val_Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(Config_ANN.filepath_accuracy_plot)
    plt.show()
    plt.close()


    plt.plot(ann_model_history.history['loss'])
    plt.plot(ann_model_history.history['val_loss'])
    plt.legend(["loss","val_loss"])
    plt.title('Loss Vs Val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(Config_ANN.filepath_loss_plot)
    plt.show()
    plt.close()

    #Predict on test set.
    predictions = ann_model.predict(test).argmax(axis=1) #directly getting classes instead of probabilities.

    #make classification report and save it directly as a file.
    report=classification_report(test_target, predictions, digits=4)
    print(report)
    with open(Config_ANN.filepath_classification_report, 'w') as report_file:
        report_file.write(report)

    print("EVERYTHING FINISHED FOR SIMPLE ANN MODEL!")

else:
    print("Simple ANN Model is not trained and evaluated")
    

## **Simple Convolutional neural Network**
CNN with one convolutional layer. The same Dense layers from Simple ANN Model are used and the first Dense Layer is replaced by a convolutional layer. No Pooling, Flattening or Dropping out. Base model for later comparison. We use a Conv1D layer, because our Input Data is one-dimensional (i.e. is a timeseries). We have sequential and not spatial data.



In [ ]:
if Train_Simple_CNN == True:
    class Config_CNN:
        epochs = 70 #70 is default (exp1)
        batch_size = 10 #10 is default (exp1)
        exp_name = 4 #Experiment Number counter 
        patience = 70 #10 # is default (exp1)
        initial_learning_rate=0.001 #Default Initial Learning Rate for ADAM: 0.001
        optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
        reduce_lr_every_10_epochs = True #Reduce the learning rate each 10 Epochs with the rate defined below. Default is False for exp1.
        lr_reduction_rate = 0.5 # Reduction Rate for reducing each 10 epochs. 0.5 means, that lr is halfed each 10 epochs.
        Conv1_filter_num = 32 # Number of filters in the convolutional layer (more means more shape-variations can be detected) 
        Conv1_filter_size = 3 # Size (e.g. 3 by 3) of single convolutional kernel. More means a more rough approach to detection of patterns.
        filepath_checkpoint = '/kaggle/working/Simple_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.weights.h5'
        filepath_accuracy_plot = '/kaggle/working/Simple_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.accuracy_plot.png'
        filepath_loss_plot = '/kaggle/working/Simple_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.loss_plot.png'
        filepath_classification_report = '/kaggle/working/Simple_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.classification_report.txt'

    #Model structure: This is not changed during experiments.
    cnn_model = tf.keras.models.Sequential()
    cnn_model.add(tf.keras.layers.Conv1D(Config_CNN.Conv1_filter_num, Config_CNN.Conv1_filter_size, activation='relu', input_shape=(187, 1))) # We add one Conv1D layer to the model
    cnn_model.add(tf.keras.layers.Flatten()) # After 
    cnn_model.add(tf.keras.layers.Dense(60, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    cnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    cnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    cnn_model.add(tf.keras.layers.Dense(5, activation='softmax'))
    
    #Function for the learning rate scheduler
    def lr_scheduler(epoch, lr):
        if Config_ANN.reduce_lr_every_10_epochs and epoch % 10 == 0: # %10 == 0 means each 10 epochs (only then no "rest" after dividing trough 10)
            return lr * Config_ANN.lr_reduction_rate  # reduce the learning rate with the configured rate
        else:
            return lr

    # callback for the learning rate reduction schedule (function)
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
    
    #Model checkpoint: Saves the best model for all epochs with regards to the validation accuracy. Only weights (.h5) are saved.
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=Config_CNN.filepath_checkpoint,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    #Early stop callback: If validation accuracy does not change during the last XXX epochs, training is stopped (XXX is configured as patience)
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=Config_CNN.patience)

    #Compilation of model with ADAM and custom lr, sparse_categorical_crossentropy since we have integers as class labels.
    cnn_model.compile(optimizer=Config_CNN.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    cnn_model.summary()

    """The following transformation of DataFrames to NumPy arrays and reshaping them is necessary to match the input requirements of the Convolutional Neural Network model.
     CNNs in TensorFlow typically expect input data in the form of NumPy arrays with specific shapes, especially when using Conv1D layers.
    This reshaping ensures that the data is in the correct format for training and inference."""
    train_array = train.to_numpy()
    test_array = test.to_numpy()
    train_reshaped = train_array.reshape(train_array.shape[0], train_array.shape[1], 1)
    test_reshaped = test_array.reshape(test_array.shape[0], test_array.shape[1], 1)
        
    #training the model and storing all results in training history. aside from the datasets, all arguments are called from our config-class. Also calling the respective callbacks.
    cnn_model_history = cnn_model.fit(train_reshaped, train_target, epochs=Config_CNN.epochs, batch_size=Config_CNN.batch_size, validation_data=(test_reshaped, test_target), callbacks=[model_checkpoint_callback, early_stop_callback, lr_scheduler_callback])
    
    #some plots for early interpretion during the run
    plt.plot(cnn_model_history.history['accuracy'])
    plt.plot(cnn_model_history.history['val_accuracy'])
    plt.legend(["accuracy","val_accuracy"])
    plt.title(f'Accuracy Vs Val_Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(Config_CNN.filepath_accuracy_plot)
    plt.show()
    plt.close()

    plt.plot(cnn_model_history.history['loss'])
    plt.plot(cnn_model_history.history['val_loss'])
    plt.legend(["loss","val_loss"])
    plt.title('Loss Vs Val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(Config_CNN.filepath_loss_plot)
    plt.show()
    plt.close()

    #Predict on test set.
    predictions = cnn_model.predict(test_reshaped).argmax(axis=1)

    #make classification report and save it directly as a file.
    report = classification_report(test_target, predictions, digits=4)
    print(report)
    with open(Config_CNN.filepath_classification_report, 'w') as report_file:
        report_file.write(report)

    print("EVERYTHING FINISHED FOR SIMPLE CNN MODEL!")

else:
    print("Simple CNN Model is not trained and evaluated")


## **Advanced CNN**

This is the Model from Hakan's notebook, it will be adapted so that the model structure can be used with exp1 to exp4.

<font color='red'>Hakan can do some notes here ################</font>

**Most important change(s):**
- Sparse_Categorical_Crossentropy instead of categorical: Should not change metrics but removes the need for one-hot encoding the dataset, which is not done in this notebook. We have numbers of 0 to 4 for each category, so it's wondering why normal categorical_cross_entropy even worked in the original notebook from Hakan?
- By using Sparse_Categorical_Crossentropy, I had to adjust the making of the classification report: test_target is now used directly and not with .argmax(axis=1). I also don't quite understand why this should be used in any way because test_target is already a set of predicted classes and doesn't need to be further processed.
- Removed the plateau learning rate callback; we can use this in experiment5, but it was not used during exp1 to 4

In [ ]:
if Train_Advanced_CNN == True:
    class Config_Advanced_CNN:
        epochs = 70 #70 is default (exp1)
        batch_size = 10 #10 is default (exp1)
        exp_name = 4 #Experiment Number counter 
        patience = 70 #10 # is default (exp1)
        initial_learning_rate=0.001 #Default Initial Learning Rate for ADAM: 0.001
        optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
        reduce_lr_every_10_epochs = True #Reduce the learning rate each 10 Epochs with the rate defined below. Default is False for exp1.
        lr_reduction_rate = 0.5 # Reduction Rate for reducing each 10 epochs. 0.5 means, that lr is halfed each 10 epochs.
        Conv1_filter_num = 32 # Number of filters in the convolutional layer (more means more shape-variations can be detected) 
        Conv1_filter_size = 3 # Size (e.g. 3 by 3) of single convolutional kernel. More means a more rough approach to detection of patterns.
        filepath_checkpoint = '/kaggle/working/Advanced_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.weights.h5'
        filepath_accuracy_plot = '/kaggle/working/Advanced_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.accuracy_plot.png'
        filepath_loss_plot = '/kaggle/working/Advanced_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.loss_plot.png'
        filepath_classification_report = '/kaggle/working/Advanced_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.classification_report.txt'

    #Model structure: This is not changed during experiments.
    adv_cnn_model = tf.keras.models.Sequential()
    adv_cnn_model.add(tf.keras.layers.Conv1D(Config_Advanced_CNN.Conv1_filter_num, Config_Advanced_CNN.Conv1_filter_size, activation='relu', input_shape=(187, 1))) # We add one Conv1D layer to the model
    adv_cnn_model.add(tf.keras.layers.MaxPooling1D(pool_size=3, strides=2)) # We add one Conv1D layer to the model
    adv_cnn_model.add(tf.keras.layers.Conv1D(Config_Advanced_CNN.Conv1_filter_num//2, Config_Advanced_CNN.Conv1_filter_size, activation='relu' )) # We add one Conv1D layer to the model
    adv_cnn_model.add(tf.keras.layers.MaxPooling1D(pool_size=3, strides=2)) # We add one Conv1D layer to the model
    adv_cnn_model.add(tf.keras.layers.Flatten()) # After  
    adv_cnn_model.add(tf.keras.layers.Dropout(rate=0.2))
    adv_cnn_model.add(tf.keras.layers.Dense(120, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    adv_cnn_model.add(tf.keras.layers.Dense(60, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    adv_cnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    adv_cnn_model.add(tf.keras.layers.Dense(5, activation='softmax'))
    
    #Function for the learning rate scheduler
    def lr_scheduler(epoch, lr):
        if Config_Advanced_CNN.reduce_lr_every_10_epochs and epoch % 10 == 0: # %10 == 0 means each 10 epochs (only then no "rest" after dividing trough 10)
            return lr * Config_Advanced_CNN.lr_reduction_rate  # reduce the learning rate with the configured rate
        else:
            return lr

    # callback for the learning rate reduction schedule (function)
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
    
    #Model checkpoint: Saves the best model for all epochs with regards to the validation accuracy. Only weights (.h5) are saved.
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=Config_Advanced_CNN.filepath_checkpoint,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    #Early stop callback: If validation accuracy does not change during the last XXX epochs, training is stopped (XXX is configured as patience)
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=Config_Advanced_CNN.patience)

    #Compilation of model with ADAM and custom lr, sparse_categorical_crossentropy since we have integers as class labels.
    adv_cnn_model.compile(optimizer=Config_Advanced_CNN.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    adv_cnn_model.summary()

    
    """The following transformation of DataFrames to NumPy arrays and reshaping them is necessary to match the input requirements of the Convolutional Neural Network model.
     CNNs in TensorFlow typically expect input data in the form of NumPy arrays with specific shapes, especially when using Conv1D layers.
    This reshaping ensures that the data is in the correct format for training and inference."""
    train_array = train.to_numpy()
    test_array = test.to_numpy()
    train_reshaped = train_array.reshape(train_array.shape[0], train_array.shape[1], 1)
    test_reshaped = test_array.reshape(test_array.shape[0], test_array.shape[1], 1)
    
    #training the model and storing all results in training history. aside from the datasets, all arguments are called from our config-class. Also calling the respective callbacks.
    adv_cnn_model_history = adv_cnn_model.fit(train_reshaped, train_target, epochs=Config_Advanced_CNN.epochs, batch_size=Config_Advanced_CNN.batch_size, 
                                      validation_data=(test_reshaped, test_target), 
                                      callbacks=[model_checkpoint_callback, early_stop_callback, lr_scheduler_callback])
    
    #some plots for early interpretion during the run
    plt.plot(adv_cnn_model_history.history['accuracy'])
    plt.plot(adv_cnn_model_history.history['val_accuracy'])
    plt.legend(["accuracy","val_accuracy"])
    plt.title(f'Accuracy Vs Val_Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(Config_Advanced_CNN.filepath_accuracy_plot)
    plt.show()
    plt.close()

    plt.plot(adv_cnn_model_history.history['loss'])
    plt.plot(adv_cnn_model_history.history['val_loss'])
    plt.legend(["loss","val_loss"])
    plt.title('Loss Vs Val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(Config_Advanced_CNN.filepath_loss_plot)
    plt.show()
    plt.close()

    #Predict on test set.
    predictions = adv_cnn_model.predict(test_reshaped).argmax(axis=1)
      
    #make classification report and save it directly as a file.
    report = classification_report(test_target, predictions, digits=4)
    print(report)
    with open(Config_Advanced_CNN.filepath_classification_report, 'w') as report_file:
        report_file.write(report)

    print("EVERYTHING FINISHED FOR ADVANCED CNN MODEL!")

else:
    print("ADVANCED CNN Model is not trained and evaluated")